# Unstructured

本 Notebook 将介绍如何使用 `Unstructured` [文档加载器](https://python.langchain.com/docs/concepts/document_loaders) 来加载多种类型的文件。`Unstructured` 目前支持文本文件、PowerPoint、HTML、PDF、图像等文件的加载。

请参阅[此指南](../../integrations/providers/unstructured.mdx)以获取有关在本地设置 Unstructured 的更多说明，包括设置所需的系统依赖项。

## 概览
### 集成详情

| 类 | 包 | 本地 | 可序列化 | [JS 支持](https://js.langchain.com/docs/integrations/document_loaders/file_loaders/unstructured/)|
| :--- | :--- | :---: | :---: |  :---: |
| [UnstructuredLoader](https://python.langchain.com/api_reference/unstructured/document_loaders/langchain_unstructured.document_loaders.UnstructuredLoader.html) | [langchain_unstructured](https://python.langchain.com/api_reference/unstructured/index.html) | ✅ | ❌ | ✅ |
### 加载器功能
| 来源 | 文档延迟加载 | 原生异步支持
| :---: | :---: | :---: |
| UnstructuredLoader | ✅ | ❌ |

## 设置

### 凭证

默认情况下，`langchain-unstructured` 会安装一个占用空间较小的版本，该版本需要将分区逻辑卸载到 Unstructured API，这需要一个 API 密钥。如果您使用本地安装，则不需要 API 密钥。要获取您的 API 密钥，请访问[此网站](https://unstructured.io)并获取 API 密钥，然后在下面的单元格中设置它：

In [1]:
import getpass
import os

if "UNSTRUCTURED_API_KEY" not in os.environ:
    os.environ["UNSTRUCTURED_API_KEY"] = getpass.getpass(
        "Enter your Unstructured API key: "
    )

### 安装

#### 标准安装

运行此 notebook 的其余部分需要以下软件包。

In [ ]:
# Install package, compatible with API partitioning
%pip install --upgrade --quiet langchain-unstructured unstructured-client unstructured "unstructured[pdf]" python-magic

#### 本地安装

如果您想在本地运行分区逻辑，您需要安装一系列系统依赖项，具体请参见 [Unstructured 文档](https://docs.unstructured.io/open-source/installation/full-installation)。

例如，在 Mac 上，您可以使用以下命令安装所需的依赖项：

```bash
# 基础依赖项
brew install libmagic poppler tesseract

# 如果解析 xml / html 文档：
brew install libxml2 libxslt
```

您可以使用以下命令安装本地运行所需的 `pip` 依赖项：

```bash
pip install "langchain-unstructured[local]"
```

## 初始化

`UnstructuredLoader` 允许从多种不同文件类型加载。要了解 `unstructured` 包的所有信息，请参阅其[文档](https://docs.unstructured.io/open-source/introduction/overview)/。在本例中，我们将展示如何从文本文件和 PDF 文件加载。

In [3]:
from langchain_unstructured import UnstructuredLoader

file_paths = [
    "./example_data/layout-parser-paper.pdf",
    "./example_data/state_of_the_union.txt",
]


loader = UnstructuredLoader(file_paths)

## Load

In [4]:
docs = loader.load()

docs[0]

INFO: pikepdf C++ to Python logger bridge initialized


Document(metadata={'source': './example_data/layout-parser-paper.pdf', 'coordinates': {'points': ((16.34, 213.36), (16.34, 253.36), (36.34, 253.36), (36.34, 213.36)), 'system': 'PixelSpace', 'layout_width': 612, 'layout_height': 792}, 'file_directory': './example_data', 'filename': 'layout-parser-paper.pdf', 'languages': ['eng'], 'last_modified': '2024-02-27T15:49:27', 'page_number': 1, 'filetype': 'application/pdf', 'category': 'UncategorizedText', 'element_id': 'd3ce55f220dfb75891b4394a18bcb973'}, page_content='1 2 0 2')

In [5]:
print(docs[0].metadata)

{'source': './example_data/layout-parser-paper.pdf', 'coordinates': {'points': ((16.34, 213.36), (16.34, 253.36), (36.34, 253.36), (36.34, 213.36)), 'system': 'PixelSpace', 'layout_width': 612, 'layout_height': 792}, 'file_directory': './example_data', 'filename': 'layout-parser-paper.pdf', 'languages': ['eng'], 'last_modified': '2024-02-27T15:49:27', 'page_number': 1, 'filetype': 'application/pdf', 'category': 'UncategorizedText', 'element_id': 'd3ce55f220dfb75891b4394a18bcb973'}


## 懒加载

In [6]:
pages = []
for doc in loader.lazy_load():
    pages.append(doc)

pages[0]

Document(metadata={'source': './example_data/layout-parser-paper.pdf', 'coordinates': {'points': ((16.34, 213.36), (16.34, 253.36), (36.34, 253.36), (36.34, 213.36)), 'system': 'PixelSpace', 'layout_width': 612, 'layout_height': 792}, 'file_directory': './example_data', 'filename': 'layout-parser-paper.pdf', 'languages': ['eng'], 'last_modified': '2024-02-27T15:49:27', 'page_number': 1, 'filetype': 'application/pdf', 'category': 'UncategorizedText', 'element_id': 'd3ce55f220dfb75891b4394a18bcb973'}, page_content='1 2 0 2')

## 后处理

如果你需要在提取 `unstructured` 元素后进行后处理，可以在实例化 `UnstructuredLoader` 时，将一个 `str` 到 `str` 的函数列表传递给 `post_processors` 关键字参数。这同样适用于其他 Unstructured 加载器。下面是一个示例。

In [7]:
from langchain_unstructured import UnstructuredLoader
from unstructured.cleaners.core import clean_extra_whitespace

loader = UnstructuredLoader(
    "./example_data/layout-parser-paper.pdf",
    post_processors=[clean_extra_whitespace],
)

docs = loader.load()

docs[5:10]

[Document(metadata={'source': './example_data/layout-parser-paper.pdf', 'coordinates': {'points': ((16.34, 393.9), (16.34, 560.0), (36.34, 560.0), (36.34, 393.9)), 'system': 'PixelSpace', 'layout_width': 612, 'layout_height': 792}, 'file_directory': './example_data', 'filename': 'layout-parser-paper.pdf', 'languages': ['eng'], 'last_modified': '2024-02-27T15:49:27', 'page_number': 1, 'parent_id': '89565df026a24279aaea20dc08cedbec', 'filetype': 'application/pdf', 'category': 'UncategorizedText', 'element_id': 'e9fa370aef7ee5c05744eb7bb7d9981b'}, page_content='2 v 8 4 3 5 1 . 3 0 1 2 : v i X r a'),
 Document(metadata={'source': './example_data/layout-parser-paper.pdf', 'coordinates': {'points': ((157.62199999999999, 114.23496279999995), (157.62199999999999, 146.5141628), (457.7358962799999, 146.5141628), (457.7358962799999, 114.23496279999995)), 'system': 'PixelSpace', 'layout_width': 612, 'layout_height': 792}, 'file_directory': './example_data', 'filename': 'layout-parser-paper.pdf', '

## 非结构化 API

如果您想快速启动小型包并获得最新的分区，您可以 `pip install
unstructured-client` 和 `pip install langchain-unstructured`。有关 `UnstructuredLoader` 的更多信息，请参阅
[Unstructured 提供商页面](https://python.langchain.com/v0.1/docs/integrations/document_loaders/unstructured_file/)。

当您传入 `api_key` 并设置 `partition_via_api=True` 时，加载器将使用托管的 Unstructured serverless API 处理您的文档。您可以在此处生成免费的
Unstructured API 密钥：[https://unstructured.io/api-key/](https://unstructured.io/api-key/)。

如果您想自托管 Unstructured API 或在本地运行它，请在此处查看说明：
[https://github.com/Unstructured-IO/unstructured-api#dizzy-instructions-for-using-the-docker-image](https://github.com/Unstructured-IO/unstructured-api#dizzy-instructions-for-using-the-docker-image)。

In [8]:
from langchain_unstructured import UnstructuredLoader

loader = UnstructuredLoader(
    file_path="example_data/fake.docx",
    api_key=os.getenv("UNSTRUCTURED_API_KEY"),
    partition_via_api=True,
)

docs = loader.load()
docs[0]

INFO: Preparing to split document for partition.
INFO: Given file doesn't have '.pdf' extension, so splitting is not enabled.
INFO: Partitioning without split.
INFO: Successfully partitioned the document.


Document(metadata={'source': 'example_data/fake.docx', 'category_depth': 0, 'filename': 'fake.docx', 'languages': ['por', 'cat'], 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'category': 'Title', 'element_id': '56d531394823d81787d77a04462ed096'}, page_content='Lorem ipsum dolor sit amet.')

您还可以通过 Unstructured API，在一次 API 请求中批量处理多个文件，使用 `UnstructuredLoader`。

In [9]:
loader = UnstructuredLoader(
    file_path=["example_data/fake.docx", "example_data/fake-email.eml"],
    api_key=os.getenv("UNSTRUCTURED_API_KEY"),
    partition_via_api=True,
)

docs = loader.load()

print(docs[0].metadata["filename"], ": ", docs[0].page_content[:100])
print(docs[-1].metadata["filename"], ": ", docs[-1].page_content[:100])

INFO: Preparing to split document for partition.
INFO: Given file doesn't have '.pdf' extension, so splitting is not enabled.
INFO: Partitioning without split.
INFO: Successfully partitioned the document.
INFO: Preparing to split document for partition.
INFO: Given file doesn't have '.pdf' extension, so splitting is not enabled.
INFO: Partitioning without split.
INFO: Successfully partitioned the document.


fake.docx :  Lorem ipsum dolor sit amet.
fake-email.eml :  Violets are blue


### 非结构化 SDK 客户端

使用非结构化 API 进行分区依赖于 [非结构化 SDK 客户端](https://docs.unstructured.io/api-reference/api-services/accessing-unstructured-api)。

如果你想自定义客户端，你需要将 `UnstructuredClient` 实例传递给 `UnstructuredLoader`。下面是一个示例，展示了如何自定义客户端的各项功能，例如使用你自己的 `requests.Session()`、传递替代的 `server_url` 以及自定义 `RetryConfig` 对象。有关自定义客户端的更多信息或 SDK 客户端接受的其他参数，请参阅 [非结构化 Python SDK](https://docs.unstructured.io/api-reference/api-services/sdk-python) 文档和 [API 参数](https://docs.unstructured.io/api-reference/api-services/api-parameters) 文档中关于客户端的部分。请注意，所有 API 参数都应传递给 `UnstructuredLoader`。

<div class="alert alert-block alert-warning"><b>警告：</b>下面的示例可能未使用最新版本的 UnstructuredClient，并且未来的版本中可能存在重大更改。有关最新示例，请参阅 <a href="https://docs.unstructured.io/api-reference/api-services/sdk-python">Unstructured Python SDK</a> 文档。</div>

In [10]:
import requests
from langchain_unstructured import UnstructuredLoader
from unstructured_client import UnstructuredClient
from unstructured_client.utils import BackoffStrategy, RetryConfig

client = UnstructuredClient(
    api_key_auth=os.getenv(
        "UNSTRUCTURED_API_KEY"
    ),  # Note: the client API param is "api_key_auth" instead of "api_key"
    client=requests.Session(),  # Define your own requests session
    server_url="https://api.unstructuredapp.io/general/v0/general",  # Define your own api url
    retry_config=RetryConfig(
        strategy="backoff",
        retry_connection_errors=True,
        backoff=BackoffStrategy(
            initial_interval=500,
            max_interval=60000,
            exponent=1.5,
            max_elapsed_time=900000,
        ),
    ),  # Define your own retry config
)

loader = UnstructuredLoader(
    "./example_data/layout-parser-paper.pdf",
    partition_via_api=True,
    client=client,
    split_pdf_page=True,
    split_pdf_page_range=[1, 10],
)

docs = loader.load()

print(docs[0].metadata["filename"], ": ", docs[0].page_content[:100])

INFO: Preparing to split document for partition.
INFO: Concurrency level set to 5
INFO: Splitting pages 1 to 10 (10 total)
INFO: Determined optimal split size of 2 pages.
INFO: Partitioning 5 files with 2 page(s) each.
INFO: Partitioning set #1 (pages 1-2).
INFO: Partitioning set #2 (pages 3-4).
INFO: Partitioning set #3 (pages 5-6).
INFO: Partitioning set #4 (pages 7-8).
INFO: Partitioning set #5 (pages 9-10).
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: Successfully partitioned set #1, elements added to the final result.
INFO: Successfully partitioned set #2, elements added to the final result.
INFO: Successfully partitioned set #3, elements added to the fi

layout-parser-paper.pdf :  LayoutParser: A Uniﬁed Toolkit for Deep Learning Based Document Image Analysis


## 分块

`UnstructuredLoader` 不支持将 `mode` 作为参数来对文本进行分组，不像旧的加载器 `UnstructuredFileLoader` 和其他加载器那样。相反，它支持“分块”。Unstructured 中的分块与您可能熟悉的、基于纯文本特征（如表示段落边界或列表项边界的字符序列 "\n\n" 或 "\n"）来形成块的其他分块机制不同。而是，所有文档都使用关于每个文档格式的特定知识进行拆分，将文档划分为语义单元（文档元素），我们只需要在单个元素超过所需的块的最大大小时进行文本拆分。总的来说，分块会合并连续的元素，形成尽可能大的块，但不会超过块的最大大。分块会生成 CompositeElement、Table 或 TableChunk 元素的序列。每个“块”是这三种类型之一的实例。

有关分块选项的更多详细信息，请参阅此[页面](https://docs.unstructured.io/open-source/core-functionality/chunking)，但要重现与 `mode="single"` 相同的行为，您可以设置 `chunking_strategy="basic"`、`max_characters=<some-really-big-number>` 和 `include_orig_elements=False`。

In [11]:
from langchain_unstructured import UnstructuredLoader

loader = UnstructuredLoader(
    "./example_data/layout-parser-paper.pdf",
    chunking_strategy="basic",
    max_characters=1000000,
    include_orig_elements=False,
)

docs = loader.load()

print("Number of LangChain documents:", len(docs))
print("Length of text in the document:", len(docs[0].page_content))

Number of LangChain documents: 1
Length of text in the document: 42772


## 加载网页

`UnstructuredLoader` 在本地运行时接受一个 `web_url` 关键字参数，该参数填充底层 Unstructured [partition](https://docs.unstructured.io/open-source/core-functionality/partitioning) 的 `url` 参数。这允许解析远程托管的文档，例如 HTML 网页。

用法示例：

In [6]:
from langchain_unstructured import UnstructuredLoader

loader = UnstructuredLoader(web_url="https://www.example.com")
docs = loader.load()

for doc in docs:
    print(f"{doc}\n")

page_content='Example Domain' metadata={'category_depth': 0, 'languages': ['eng'], 'filetype': 'text/html', 'url': 'https://www.example.com', 'category': 'Title', 'element_id': 'fdaa78d856f9d143aeeed85bf23f58f8'}

page_content='This domain is for use in illustrative examples in documents. You may use this domain in literature without prior coordination or asking for permission.' metadata={'languages': ['eng'], 'parent_id': 'fdaa78d856f9d143aeeed85bf23f58f8', 'filetype': 'text/html', 'url': 'https://www.example.com', 'category': 'NarrativeText', 'element_id': '3652b8458b0688639f973fe36253c992'}

page_content='More information...' metadata={'category_depth': 0, 'link_texts': ['More information...'], 'link_urls': ['https://www.iana.org/domains/example'], 'languages': ['eng'], 'filetype': 'text/html', 'url': 'https://www.example.com', 'category': 'Title', 'element_id': '793ab98565d6f6d6f3a6d614e3ace2a9'}



## API 参考

有关 `UnstructuredLoader` 的所有功能和配置的详细文档，请访问 API 参考：https://python.langchain.com/api_reference/unstructured/document_loaders/langchain_unstructured.document_loaders.UnstructuredLoader.html